## packages imports

In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
import ydata_profiling
from ydata_profiling import ProfileReport
import great_expectations as gx

In [ ]:
def save_expectations(expectation_suite, file_name:str):
    with open(file_name+".json", "w") as my_file:
        my_file.write(
            json.dumps(expectation_suite.to_json_dict(), sort_keys=True, indent=4)
        )

## data imports

In [ ]:
data = pd.read_csv("merged_epc_mcs_installations_installers_230906.csv")

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
data["HP_INSTALLED"].value_counts(dropna=False)

# EXPECTATIONS

## 1. Data where HP_INSTALLED is False

In [ ]:
no_hp_installed = data[data["HP_INSTALLED"]==False]
no_hp_installed = gx.from_pandas(no_hp_installed)

When `HP_INSTALLED` is `False` (there's an EPC but no HP installed):
- `MCS_AVAILABLE` should always be `False`
- `HP_TYPE` should always be `No HP`

In [ ]:
no_hp_installed.expect_column_distinct_values_to_equal_set(column = "MCS_AVAILABLE", value_set = [False])["success"]

In [ ]:
no_hp_installed.expect_column_values_to_not_be_null(column = "MCS_AVAILABLE")["success"]

In [ ]:
no_hp_installed.expect_column_distinct_values_to_equal_set(column = "HP_TYPE", value_set = ["No HP"])["success"]

In [ ]:
no_hp_installed.expect_column_values_to_not_be_null(column = "HP_TYPE")["success"]

In [ ]:
no_hp_installed_expectation_suite = no_hp_installed.get_expectation_suite()

In [ ]:
save_expectations(no_hp_installed_expectation_suite, "no_hp_installed_expectation_suite")

In [ ]:
del no_hp_installed_expectation_suite
del no_hp_installed

## 2. Data where HP_INSTALLED is missing

In [ ]:
just_installers = data[pd.isnull(data["HP_INSTALLED"])]
just_installers = gx.from_pandas(just_installers)

In [ ]:
just_installers["MCS_AVAILABLE"].unique()

In [ ]:
just_installers.expect_column_values_to_be_null(column = "MCS_AVAILABLE")["success"]

In [ ]:
just_installers.expect_column_values_to_be_null(column = "EPC_AVAILABLE")["success"]

In [ ]:
just_installers.expect_column_values_to_be_null(column = "HP_TYPE")["success"]

In [ ]:
just_installers_expectation_suite = just_installers.get_expectation_suite()

In [ ]:
save_expectations(just_installers_expectation_suite, "just_installers_expectation_suite")

In [ ]:
del just_installers_expectation_suite
del just_installers

## 3. Where HP_INSTALLED is True

In [ ]:
hp_data = data[data["HP_INSTALLED"]==True]
hp_data = gx.from_pandas(hp_data)

In [ ]:
del data

### profile report (helpful to define expectations)

In [ ]:
profile_report = ProfileReport(df=hp_data, title="HP data")

In [ ]:
profile_report.to_notebook_iframe()

### Variables we expect: (we have more than these, but not all of them are used in HPMT)

I've just added the ones in the HPMT workbook:

In [ ]:
hpmt_vars = ["design",
"flow_temp",
"HP_TYPE",
"capacity",
"estimated_annual_generation",
"scop",
"manufacturer",
"product_name",
"HP_INSTALLED",
"cost",
"HP_INSTALL_DATE",
"effective_to",
"effective_from",
"latitude",
"longitude",
"air_source_hp_certified",
"exhaust_air_hp_certified",
"ground_water_source_hp_certified",
"company_unique_id",
"company_name",
"BUILT_FORM",
"CONSTRUCTION_AGE_BAND",
"FLOOR_ENERGY_EFF",
"HOT_WATER_ENERGY_EFF",
"LIGHTING_ENERGY_EFF",
"MAINHEAT_ENERGY_EFF",
"ROOF_ENERGY_EFF",
"WALLS_ENERGY_EFF",
"WINDOWS_ENERGY_EFF",
"CURRENT_ENERGY_RATING",
"POTENTIAL_ENERGY_RATING",
"LATITUDE",
"LONGITUDE",
"COUNTRY",
"POSTCODE",
"GLAZED_AREA",
"GLAZED_TYPE",
"HEATING_FUEL",
"HEATING_SYSTEM",
"INSPECTION_DATE",
"MAINS_GAS_FLAG",
"MCS_AVAILABLE",
"EPC_AVAILABLE",
"NUMBER_HABITABLE_ROOMS",
"PHOTO_SUPPLY",
"SOLAR_WATER_HEATING_FLAG",
"TENURE",
"TOTAL_FLOOR_AREA",
"PROPERTY_TYPE",
"UPRN"]

In [ ]:
hp_data.expect_table_columns_to_match_set(hpmt_vars, exact_match=False)["success"]

### Expected unique values for a variety of variables:

`HP_INSTALLED` is always `True`:

In [ ]:
hp_data.expect_column_distinct_values_to_equal_set(column = "HP_INSTALLED", value_set = [True])["success"]

`HEATING_SYSTEM` is always `heat pump`:

In [ ]:
hp_data.expect_column_distinct_values_to_equal_set(column = "HEATING_SYSTEM", value_set = ["heat pump"])["success"]

`HEATING_FUEL` is always `electric`:

In [ ]:
hp_data.expect_column_distinct_values_to_equal_set(column = "HEATING_FUEL", value_set = ["electric"])["success"]

In [ ]:
hp_data.expect_column_distinct_values_to_equal_set(column = "MCS_AVAILABLE", value_set = [True, False])["success"]

In [ ]:
hp_data.expect_column_distinct_values_to_equal_set(column = "EPC_AVAILABLE", value_set = [True, False])["success"]

In [ ]:
hp_data.expect_column_distinct_values_to_equal_set(column = "MAINS_GAS_FLAG", value_set = ["N", "Y"])["success"]

In [ ]:
hp_data.expect_column_distinct_values_to_equal_set(column = "SOLAR_WATER_HEATING_FLAG", value_set = [True, False])["success"]

In [ ]:
for var in ["CURRENT_ENERGY_RATING", "POTENTIAL_ENERGY_RATING"]:
    hp_data.expect_column_distinct_values_to_equal_set(column = var, value_set = ["A", "B", "C", "D", "E", "F", "G"])["success"]

In [ ]:
for var in ["WALLS_ENERGY_EFF", "ROOF_ENERGY_EFF", "FLOOR_ENERGY_EFF", "WINDOWS_ENERGY_EFF", "MAINHEAT_ENERGY_EFF", "MAINHEATC_ENERGY_EFF", "HOT_WATER_ENERGY_EFF", "LIGHTING_ENERGY_EFF"]:
    hp_data.expect_column_distinct_values_to_equal_set(column = var, value_set = ['Very Good', 'Good', 'Average', 'Poor', 'Very Poor'])["success"]

In [ ]:
hp_data.expect_column_distinct_values_to_equal_set(column = "CONSTRUCTION_AGE_BAND", value_set = ['England and Wales: before 1900',
                                                                                                  'Scotland: before 1919',
                                                                                                  '1900-1929',
                                                                                                  '1930-1949',
                                                                                                  '1950-1966',
                                                                                                  '1965-1975',
                                                                                                  '1976-1983',
                                                                                                  '1983-1991',
                                                                                                  '1991-1998',
                                                                                                  '1996-2002',
                                                                                                  '2003-2007',
                                                                                                  '2007 onwards'])["success"]

In [ ]:
hp_data.expect_column_distinct_values_to_equal_set(column = "GLAZED_AREA", value_set = [1,2,3,4,5])["success"]

In [ ]:
hp_data.expect_column_distinct_values_to_equal_set(column = "GLAZED_TYPE", value_set = ['double glazing', 'triple glazing', 'single glazing'])["success"]

In [ ]:
hp_data.expect_column_distinct_values_to_equal_set(column = "TENURE", value_set = ['rental (private)', 'rental (social)', 'owner-occupied'])["success"]

In [ ]:
hp_data.expect_column_distinct_values_to_equal_set(column = "BUILT_FORM", value_set = ['Semi-Detached',
                                                                                       'Detached',
                                                                                       'Mid-Terrace',
                                                                                       'Enclosed Mid-Terrace',
                                                                                       'End-Terrace',
                                                                                       'Enclosed End-Terrace'])["success"]

In [ ]:
hp_data.expect_column_distinct_values_to_equal_set(column = "PROPERTY_TYPE", value_set = ['Flat',
                                                                                       'House',
                                                                                       'Bungalow',
                                                                                       'Maisonette',
                                                                                       'Park home'])["success"]

In [ ]:
hp_data.expect_column_distinct_values_to_equal_set(column = "COUNTRY", value_set = ['England', 'Wales', 'Scotland'])["success"]

In [ ]:
hp_data.expect_column_distinct_values_to_equal_set(column = "HP_TYPE", value_set = ['Air Source Heat Pump',
                                                                                    'Ground/Water Source Heat Pump',
                                                                                    'Undefined or Other Heat Pump Type'])["success"]

In [ ]:
hp_data.expect_column_distinct_values_to_equal_set(column = "installation_type", value_set = ['Domestic'])["success"]

In [ ]:
hp_certifified_cols = [col for col in hp_data.columns if col.endswith("certified")]
for col in hp_certifified_cols:
    hp_data.expect_column_distinct_values_to_equal_set(column = col, value_set = [True, False])["success"]

In [ ]:
hp_data.expect_column_distinct_values_to_equal_set(column = "design", value_set = ['Space heat and DHW',
                                                                                   'Space heat only',
                                                                                   'Space Heat, DHW and another purpose',
                                                                                   'DHW only',
                                                                                   'DHW and another purpose',
                                                                                   'Space Heat and another purpose',
                                                                                   'Another purpose only'])["success"]

### Variables with values in a certain range:

In [ ]:
hp_data.expect_column_values_to_be_between(column = "NUMBER_HABITABLE_ROOMS", min_value = 0 , strict_min = False)["success"]

In [ ]:
hp_data.expect_column_values_to_be_between(column = "NUMBER_HEATED_ROOMS", min_value = 0 , strict_min = False)["success"]

In [ ]:
hp_data.expect_column_values_to_be_between(column = "ENERGY_CONSUMPTION_CURRENT", min_value = 0 , strict_min = False)["success"]

In [ ]:
hp_data.expect_column_values_to_be_between(column = "TOTAL_FLOOR_AREA", min_value = 0 , strict_min = False)["success"]

In [ ]:
hp_data.expect_column_values_to_be_between(column = "CURRENT_ENERGY_EFFICIENCY", min_value = 0 , strict_min = True)["success"]

In [ ]:
hp_data.expect_column_values_to_be_between(column = "PHOTO_SUPPLY", min_value = 0 , max_value = 100, strict_min = False, strict_max = False)["success"]

In [ ]:
hp_data.expect_column_values_to_be_between(column = "capacity", min_value = 0, strict_min = False)["success"]

In [ ]:
hp_data.expect_column_values_to_be_between(column = "estimated_annual_generation", min_value = 0, strict_min = False)["success"]

In [ ]:
hp_data.expect_column_values_to_be_between(column = "cost", min_value = 0, strict_min = True)["success"]

In [ ]:
hp_data.expect_column_values_to_be_between(column = "flow_temp", min_value = 0, strict_min = True)["success"]

In [ ]:
hp_data.expect_column_values_to_be_between(column = "scop", min_value = 0, strict_min = True)["success"]

### Columns with unique values:

In [ ]:
hp_data.expect_column_values_to_be_unique(column="UPRN")["success"]

### Columns with no missing data:

In [ ]:
hp_data.expect_column_values_to_not_be_null(column = "HP_INSTALLED")["success"]

In [ ]:
hp_data.expect_column_values_to_not_be_null(column = "HEATING_SYSTEM")["success"]

In [ ]:
hp_data.expect_column_values_to_not_be_null(column = "HEATING_FUEL")["success"]

In [ ]:
hp_data.expect_column_values_to_not_be_null(column = "HP_TYPE")["success"]

In [ ]:
hp_data.expect_column_values_to_not_be_null(column = "MCS_AVAILABLE")["success"]

In [ ]:
hp_data.expect_column_values_to_not_be_null(column = "EPC_AVAILABLE")["success"]

### Columns with small percentage of missing data:

In [ ]:
hp_data.expect_column_values_to_not_be_null(column = "POSTCODE", mostly = 0.99)["success"]

In [ ]:
hp_data.expect_column_values_to_not_be_null(column = "HP_INSTALL_DATE", mostly = 0.99)["success"]

Missing values for `COUNTRY` are not really missing. The missing come mostly from `MCS_AVAILABLE` data. asf-daps outputs should have as many missing as `POSTCODE` (<0.1%):

In [ ]:
hp_data.expect_column_values_to_not_be_null(column = "COUNTRY", mostly = 0.99)["success"]

In [ ]:
hp_data[pd.isnull(hp_data["COUNTRY"])]["MCS_AVAILABLE"].value_counts()

In [ ]:
hp_data[pd.isnull(hp_data["COUNTRY"])]["EPC_AVAILABLE"].value_counts()

In [ ]:
hp_data_expectation_suite = hp_data.get_expectation_suite()

In [ ]:
save_expectations(hp_data_expectation_suite, "hp_data_expectation_suite")

In [ ]:
del hp_data_expectation_suite

## 4. MCS specific checks

In [ ]:
mcs_hp_data = hp_data[hp_data["MCS_AVAILABLE"]]

In [ ]:
most_mcs_columns = ['capacity',
       'estimated_annual_generation', 'installer_name', 'design', 'cost',
        'product_name', 'manufacturer','company_unique_id', 'company_name', 'latitude', 'longitude',
       'effective_from', 'effective_to', 'air_source_hp_certified',
       'ground_water_source_hp_certified', 'exhaust_air_hp_certified']

In [ ]:
for col in most_mcs_columns:
    mcs_hp_data.expect_column_values_to_not_be_null(column = col, mostly = 0.9)["success"]

In [ ]:
for col in ['flow_temp','scop']:
    mcs_hp_data.expect_column_values_to_not_be_null(column = col, mostly = 0.84)["success"]

In [ ]:
# These are not really "missing values" - they correspond to installation type "Unspecified"
mcs_hp_data.expect_column_values_to_not_be_null(column = 'installation_type', mostly = 0.85)["success"]

In [ ]:
mcs_hp_data_expectation_suite = mcs_hp_data.get_expectation_suite()

In [ ]:
save_expectations(mcs_hp_data_expectation_suite, "mcs_hp_data_expectation_suite")

In [ ]:
del mcs_hp_data_expectation_suite
del mcs_hp_data

## 5. EPC and MCS specific data checks:

Just making sure we don't have missings due to how we're merging the two datasets (if we find a value of 100 that would be the case).

In [ ]:
epc_mcs_hp_data = hp_data[hp_data["EPC_AVAILABLE"] & hp_data["MCS_AVAILABLE"]]

In [ ]:
epc_mcs_percentages = pd.DataFrame(data =epc_mcs_hp_data.isnull().sum()/len(epc_mcs_hp_data)*100)
epc_mcs_percentages.columns = ["percentage"]

In [ ]:
epc_mcs_percentages = gx.from_pandas(epc_mcs_percentages)

In [ ]:
epc_mcs_percentages.expect_column_values_to_be_between(column = "percentage", max_value = 90, strict_max = False)["success"]

In [ ]:
mcs_epc_hp_data_expectation_suite = epc_mcs_percentages.get_expectation_suite()

In [ ]:
save_expectations(mcs_epc_hp_data_expectation_suite, "mcs_epc_hp_data_expectation_suite")

In [ ]:
del mcs_epc_hp_data_expectation_suite
del epc_mcs_percentages

## 6. EPC specific data checks:

In [ ]:
epc_hp_data = hp_data[hp_data["EPC_AVAILABLE"]]

In [ ]:
epc_percentages = pd.DataFrame(epc_hp_data.isnull().sum()/len(epc_hp_data)*100)
epc_percentages.columns = ["percentage"]

In [ ]:
epc_percentages = gx.from_pandas(epc_percentages)

In [ ]:
epc_percentages.expect_column_values_to_be_between(column = "percentage", max_value = 90, strict_max = False)["success"]

In [ ]:
epc_hp_data_expectation_suite = epc_percentages.get_expectation_suite()

In [ ]:
save_expectations(epc_hp_data_expectation_suite, "epc_hp_data_expectation_suite")

In [ ]:
del epc_hp_data_expectation_suite
del epc_percentages

## 7. Based on batch data - useful for comparisons between asf_core_data and asf_daps data

In [ ]:
hp_data_asf_daps = hp_data.copy()

In [ ]:
del hp_data

In [ ]:
hp_data_asf_daps.expect_table_row_count_to_equal(value = len(hp_data_asf_daps))["success"]

In [ ]:
for col in ["UPRN", "POSTCODE", "HP_INSTALL_DATE", "effective_from", "effective_to"]:
    hp_data_asf_daps.expect_column_unique_value_count_to_be_between(column = col, min_value = hp_data_asf_daps[col].nunique(),
                                                               max_value = hp_data_asf_daps[col].nunique())["success"]

In [ ]:
for col in ["TOTAL_FLOOR_AREA", "CURRENT_ENERGY_EFFICIENCY", "NUMBER_HABITABLE_ROOMS", "capacity",
           "estimated_annual_generation", "cost", "flow_temp", "scop", "latitude", "longitude",
           "LATITUDE", "LONGITUDE"]:
    hp_data_asf_daps.expect_column_values_to_be_between(column = col, min_value = hp_data_asf_daps[col].nunique(),
                                                               max_value = hp_data_asf_daps[col].nunique())["success"]

In [ ]:
hp_data_asf_daps_expectation_suite = hp_data_asf_daps.get_expectation_suite()

In [ ]:
save_expectations(hp_data_asf_daps_expectation_suite, "hp_data_asf_daps_expectation_suite")

In [ ]:
del hp_data_asf_daps_expectation_suite
del hp_data_asf_daps